In [ ]:
! pip install tensorflow

In [ ]:
#importing libraries
import sklearn
import pandas as pd
import seaborn as sns
import keras as K
import keras.layers as Dense
import keras.models as Sequential
import keras.optimizers as Adam
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
data_location = 'sqlite:///../data_v2/avocado.db'
data = pd.read_sql('SELECT * FROM avocado', data_location)

In [ ]:
data.head(10)

In [ ]:
data = data.iloc[: , 1:]

In [ ]:
#seperating the prices to be predicted
y = data.AveragePrice
data.drop(['AveragePrice'], axis=1, inplace=True)

In [ ]:
data = data.astype({'Date': 'object', 'Total Volume' : 'float64', '4046' : 'float64', '4225' : 'float64', '4770' : 'float64', 'Total Bags' : 'float64', 'Small Bags' : 'float64', 'Large Bags' : 'float64', 'XLarge Bags' : 'float64', 'type' : 'object', 'year' : 'int', 'region':'object' })

In [ ]:
#train-test split
#splitting the data into training and test datasets
from sklearn.model_selection import train_test_split

trainflights, testflights, ytrain, ytest = train_test_split(data, y, train_size=0.7,test_size=0.3, random_state=0)

In [ ]:
s = (trainflights.dtypes == 'object')
object_cols = list(s[s].index)

n = (trainflights.dtypes == ('float64','int64'))
numerical_cols = list(n[n].index)

In [ ]:
#checking the columns containing categorical columns:
print(object_cols)

In [ ]:
#using One Hot Encoder to make the categorical columns usable

oneHot = OneHotEncoder(handle_unknown = 'ignore', sparse=False)
oneHottrain = pd.DataFrame(oneHot.fit_transform(trainflights[object_cols]))
oneHottest = pd.DataFrame(oneHot.transform(testflights[object_cols]))

#reattaching index since OneHotEncoder removes them:
oneHottrain.index = trainflights.index
oneHottest.index = testflights.index 

#dropping the old categorical columns:
cattraincol = trainflights.drop(object_cols, axis=1)
cattestcol = testflights.drop(object_cols, axis=1)

#concatenating the new columns:
trainflights = pd.concat([cattraincol, oneHottrain], axis=1)
testflights = pd.concat([cattestcol, oneHottest], axis=1)

In [ ]:
#scaling the values

trainf = trainflights.values
testf = testflights.values

minmax = MinMaxScaler()

trainflights = minmax.fit_transform(trainf)
testflights = minmax.transform(testf)

#defining a way to find Mean Absolute Percentage Error:
def PercentError(preds, ytest):
  error = abs(preds - ytest)

  errorp = np.mean(100 - 100*(error/ytest))

  print('the accuracy is:', errorp)

In [ ]:
#implementing the algo:
model = RandomForestRegressor(n_estimators=100, random_state=0, verbose=1)

#fitting the data to random forest regressor:
model.fit(trainflights, ytrain)

In [ ]:
#predicting the test dataset:
preds = model.predict(testflights)


In [ ]:
ytest = ytest.astype('float')

In [ ]:
PercentError(preds, ytest)

In [ ]:
#using linear regression:
LinearModel = LinearRegression()
LinearModel.fit(trainflights, ytrain)

In [ ]:
#predicting on the test dataset:
LinearPredictions = LinearModel.predict(testflights)
PercentError(LinearPredictions, ytest)

I got to this point but I think that train_test_split is not allowed to be used on time series data. I am pretty sure that this is not time series data but it is very similar region wise I guess. Let's just assume that data is not time correlated in any way.

On the other hand how can it not be correlated? As we know what the price will be in August and in December, predicting the price in the October is much easier that way, that is why maybe just going for the train test split scews the correctness of the predictions by a long shot. What if I use TimeSeriesSplit instead?TimeSeriesSplit